In [1]:
import os
import time
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import FinNews as fn
import re

# Dash modules
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

# Set up jupyter proxy
# JupyterDash.infer_jupyter_proxy_config() 

%matplotlib inline

In [2]:
# Getting all file paths

path = r'assets/historical-symbols' # use your path
all_files = glob.glob(path + "/*.csv")

# Creating list to append all ticker dfs to
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concat all ticker dfs
stock_df = pd.concat(li, axis=0, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Creating Moving Average Technical Indicator
# Using this aritcle https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb

stock_df['SMA_5'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 5).mean())
stock_df['SMA_15'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 15).mean())
stock_df['SMA_ratio'] = stock_df['SMA_15'] / stock_df['SMA_5']

# Bollinger bands
stock_df['15MA'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).mean())
stock_df['SD'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).std())
stock_df['upperband'] = stock_df['15MA'] + 2*stock_df['SD']
stock_df['lowerband'] = stock_df['15MA'] - 2*stock_df['SD']

available_indicators1 = list(stock_df['ticker'].unique())

In [6]:
stock_df.groupby(['sector','Date']).mean()['Close'].reset_index()

,sector,Date,Close
0,Basic Materials,1972-06-01,1.155741
1,Basic Materials,1972-06-02,1.168303
2,Basic Materials,1972-06-05,1.165163
3,Basic Materials,1972-06-06,1.162022
4,Basic Materials,1972-06-07,1.141608
...,...,...,...
156739,Utilities,2021-06-28,73.684783
156740,Utilities,2021-06-29,72.436675
156741,Utilities,2021-06-30,72.357436
156742,Utilities,2021-07-01,73.151881


In [3]:
stock_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_adj,High_adj,...,Adj Close,sector,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
0,1990-02-16,0.000000,0.058887,0.054407,0.056967,940636800.0,0.0,0.0,0.000000,0.079861,...,0.056967,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-02-20,0.000000,0.058887,0.055047,0.058887,151862400.0,0.0,0.0,0.000000,0.079861,...,0.058887,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-02-21,0.000000,0.058247,0.055687,0.057607,70531200.0,0.0,0.0,0.000000,0.078993,...,0.057607,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-02-22,0.000000,0.060167,0.058247,0.058247,45216000.0,0.0,0.0,0.000000,0.081597,...,0.058247,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-02-23,0.000000,0.058887,0.057607,0.057927,44697600.0,0.0,0.0,0.000000,0.079861,...,0.057927,Technology,CSCO,0.057927,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002527,2021-06-28,28.400000,28.430000,28.160000,28.250000,3125600.0,0.0,0.0,28.400000,28.430000,...,28.250000,Utilities,PPL,28.052000,28.514333,1.016481,28.514333,0.576358,29.667049,27.361618
4002528,2021-06-29,28.200001,28.350000,27.680000,27.770000,3851200.0,0.0,0.0,28.200001,28.350000,...,27.770000,Utilities,PPL,28.008000,28.466667,1.016376,28.466667,0.607673,29.682013,27.251321
4002529,2021-06-30,27.840000,28.030001,27.750000,27.969999,3990000.0,0.0,0.0,27.840000,28.030001,...,27.969999,Utilities,PPL,28.044000,28.398667,1.012647,28.398667,0.601971,29.602609,27.194724
4002530,2021-07-01,28.059999,28.370001,27.879999,28.260000,4416200.0,0.0,0.0,28.059999,28.370001,...,28.260000,Utilities,PPL,28.110000,28.344000,1.008324,28.344000,0.572174,29.488347,27.199653


In [12]:
# Exploring news
import FinNews as fn

topics = 'Technology'.lower().split(" ")

seeking_alpha = fn.SeekingAlpha(topics=topics)

seeking_alpha.get_news()

[{'title': "Antiques platform 1stDibs' stock rebounds nearly 30% from last week’s post-IPO low",
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://seekingalpha.com/sector/technology.xml',
   'value': "Antiques platform 1stDibs' stock rebounds nearly 30% from last week’s post-IPO low"},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://seekingalpha.com/news/3723563-1stdibs-stock-rebounds--from-post-ipo-low?source=feed_sector_technology'}],
  'link': 'https://seekingalpha.com/news/3723563-1stdibs-stock-rebounds--from-post-ipo-low?source=feed_sector_technology',
  'id': '3723563',
  'guidislink': False,
  'published': 'Mon, 02 Aug 2021 18:06:17 -0400',
  'published_parsed': time.struct_time(tm_year=2021, tm_mon=8, tm_mday=2, tm_hour=22, tm_min=6, tm_sec=17, tm_wday=0, tm_yday=214, tm_isdst=0),
  'tags': [{'term': 'DIBS', 'scheme': None, 'label': None}],
  'topic': 'technology',
  'tag_terms': 'DIBS'},
 {'title': 'Harmonic Inc. 2

In [16]:
seeking_alpha.get_current_topics()

['technology']

In [15]:
help(seeking_alpha)

Help on SeekingAlpha in module FinNews.sources object:

class SeekingAlpha(FinNews.source_object.Source)
 |  SeekingAlpha(topics=[], save_feeds=True)
 |  
 |  Method resolution order:
 |      SeekingAlpha
 |      FinNews.source_object.Source
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, topics=[], save_feeds=True)
 |      Object for maintaining Seeking Alpha rss feeds.
 |      topics: a list of rss feed topics, must be one of the possible topics
 |          You can leave the list blank and call CNBC.get_possible_topics() and then add topics using CNBC.add_topics()
 |      save_feeds: Feed objects can save all previous news entries if this is True, otherwise the object will only the newest entries
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from FinNews.source_object.Source:
 |  
 |  add_feed(self, url, source_name, topic_name)
 |      Allows you to add a feed from a url not provided or from a dif

In [17]:
title_style = {'display': 'inline-block',
               'textAlign':'left',
               'verticalAlign':'center',
               'lineHeight':2.5,
               'height':75,
               'border': 'thick black solid',\
               'width':'85%',
               'backgroundColor': 'rgb(212, 150, 18)'}

title_link_style = {'display': 'inline-block',
               'textAlign':'center',
               'vertical-align':'center',
               'lineHeight':5,
               'height':75,
               'width':'10%',
               'float':'right',
               'border': 'thick black solid',\
               'fontSize':12,
               'backgroundColor': 'rgb(212, 150, 18)'}


tab_style = {'display': 'inline-block',
             'textAlign':'center',
             'verticalAlign':'top',
             'width':'15%',
             'border': 'thin lightgrey solid',
             'height':20}

blank_tab_style = {'display': 'inline-block',
                   'width':'15%',
                   'textAlign':'center',
                   'verticalAlign':'top',
                   'float':'right',
                   'border': 'thin lightgrey solid',
                   'fontSize':10,
                   'height':20}

ticker_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300}

chart_style = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':300}

portfolio_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':300}

chart_style_b = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':200}

portfolio_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'fontSize':12,
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':200}

news_style = {'fontSize':11,
              'color':'white',
              'textAlign':'left',
              'height':'auto',
#               'height':27,
              'margin':3,
              "lineHeight":1.5,
              'float':'middle'}

news_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300,'margin':0,'overflowY':'scroll',
                'color':'white',
                'backgroundColor':'black',
                'borderRadius': '.4rem'}



#                'vertical-align':'middle',

In [18]:
app = JupyterDash(__name__,suppress_callback_exceptions=True,external_stylesheets=[dbc.themes.SUPERHERO])

server = app.server

app.layout = html.Div([

    html.Div([
        html.H1('Dashboard Title',style=title_style),
        html.H3('Dashboard Info Link',style=title_link_style)
        ]),

    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Portfolio Performance', value='tab-1'),
        dcc.Tab(label='Model Performance', value='tab-2'),
        dcc.Tab(label='Tab three', value='tab-3')
    ],style={'width':'50%','height':50}),
    html.Div(id='tabs-example-content')
])

@app.callback(dash.dependencies.Output('tabs-example-content', 'children'),
              [dash.dependencies.Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
                html.Div([
                    dcc.Dropdown(id='ticker',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value='CSCO') # the default is code_module AAA
                    ]),
                
                html.Div([
                    html.H2('Portfolio Performance',style=portfolio_style),
                    dcc.Graph(id='price_chart',style=chart_style),
                    html.Div(id='news_list',children=news_info,style=news_style_b)
                    ]),
                    
                html.Div([
                    html.H2('Other Portfolio Statistics',style=portfolio_style_b),
                    html.H2('News Info',style=chart_style_b),
                    ],style={'height':200})
    
                ])
                
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    
# Callback to connect input(s) to output(s)
@app.callback(dash.dependencies.Output('price_chart','figure'),
    [dash.dependencies.Input('ticker','value')])

# Step 3: Define the graph with plotly express
def update_ticker(ticker):
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']==ticker]['Date'],
                             y=stock_df[stock_df['ticker']==ticker]['Close'],
                            line={"color": "#228B22"},
                            mode="lines"))

    fig.update_layout(title_text=f'{ticker} Closing Price',title_x=0.5,
                         template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                         yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

    return fig

# Creating callback to get news when ticker changes
@app.callback(dash.dependencies.Output('news_list', 'children'),
              [dash.dependencies.Input('ticker', 'value')])
def update_news(ticker):
    
    seeking_alpha = fn.SeekingAlpha(topics=['$'+ticker], save_feeds=True)
    
    news = seeking_alpha.get_news()
    
#     title_strings = [re.sub('[^A-Za-z0-9,\s]+', '', item['title']).lower().replace(" ", "-") for item in news[:5]]
#     url_ids = [re.sub('MarketCurrent:','news/',item) for item in news[:5]]
    
    news_info = html.Div(f'News for {ticker}',style={'backgroundColor':'gray'}),\
                    dbc.ListGroup([
                        dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[0]['title'],style=news_style),\
                                        href=(news[0]['link'])),\
                                        html.A(html.P(news[0]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[1]['title'],style=news_style),\
                                        href=(news[1]['link'])),\
                                        html.A(html.P(news[1]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[2]['title'],style=news_style),\
                                        href=(news[2]['link'])),\
                                        html.A(html.P(news[2]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[3]['title'],style=news_style),\
                                        href=(news[3]['link'])),\
                                        html.A(html.P(news[3]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[4]['title'],style=news_style),\
                                        href=(news[4]['link'])),\
                                        html.A(html.P(news[4]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    ],flush=True)
                    
    return news_info

if __name__ == '__main__':
    app.run_server(debug=True,port=8055)

Dash app running on http://127.0.0.1:8055/


### Notes
1. How do I set a relative height
2. Could add ticker - https://community.plotly.com/t/strip-ticker-label/48348/6


In [20]:
html.Div([
        # Adding drop down to filter by ticker
        dcc.Dropdown(id='ticker',
            options=[{'label': i, 'value': i} for i in list(stock_df['ticker'].unique())],
            value='CSCO',style={'margin':'5px','width':'40%'}), # the default is code_module AAA

        dcc.Dropdown(id='industry_ticker',
            options=[{'label': i, 'value': i} for i in list(stock_df['sector'].unique())],
            value='Technology',style={'margin':'5px','width':'40%'}) # the default is code_module AAA
            ])

Div([Dropdown(id='ticker', options=[{'label': 'CSCO', 'value': 'CSCO'}, {'label': 'UAL', 'value': 'UAL'}, {'label': 'TROW', 'value': 'TROW'}, {'label': 'ISRG', 'value': 'ISRG'}, {'label': 'NVR', 'value': 'NVR'}, {'label': 'PRGO', 'value': 'PRGO'}, {'label': 'TPR', 'value': 'TPR'}, {'label': 'DVN', 'value': 'DVN'}, {'label': 'CE', 'value': 'CE'}, {'label': 'MRO', 'value': 'MRO'}, {'label': 'BA', 'value': 'BA'}, {'label': 'VRTX', 'value': 'VRTX'}, {'label': 'GILD', 'value': 'GILD'}, {'label': 'NLSN', 'value': 'NLSN'}, {'label': 'EQIX', 'value': 'EQIX'}, {'label': 'TER', 'value': 'TER'}, {'label': 'MDT', 'value': 'MDT'}, {'label': 'V', 'value': 'V'}, {'label': 'QRVO', 'value': 'QRVO'}, {'label': 'A', 'value': 'A'}, {'label': 'FOX', 'value': 'FOX'}, {'label': 'FLT', 'value': 'FLT'}, {'label': 'MO', 'value': 'MO'}, {'label': 'SWKS', 'value': 'SWKS'}, {'label': 'ENPH', 'value': 'ENPH'}, {'label': 'MCHP', 'value': 'MCHP'}, {'label': 'CDNS', 'value': 'CDNS'}, {'label': 'WLTW', 'value': 'WLTW'}

In [22]:
[{'label': i, 'value': i} for i in ['Stocks','Sector']]

[{'label': 'Stocks', 'value': 'Stocks'},
 {'label': 'Sector', 'value': 'Sector'}]

In [6]:
stock_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,sector,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
0,1985-07-22,0.0,4.12,3.84,4.12,59400.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-07-23,0.0,4.26,4.07,4.12,20800.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-07-24,0.0,4.07,3.94,4.03,32600.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-07-25,0.0,4.07,4.03,4.07,16600.0,0.0,0.0,Energy,DVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-07-26,0.0,4.07,3.98,4.03,10400.0,0.0,0.0,Energy,DVN,4.074,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']=='AAPL']['Date'],
                         y=stock_df[stock_df['ticker']=='AAPL']['Close'],
                        line={"color": "#228B22"},
                        mode="lines"))

fig.update_layout(title_text='AAPL Closing Price',title_x=0.5,
                     template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                     yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

In [72]:
df = df.set_index('Date')
df

,Close,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
Date,,,,,,,,,,,,,,,,,
1980-12-12,0.100751,0.0,0.101189,0.100751,0.100751,0.0,469033600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-15,0.095495,0.0,0.095933,0.095495,0.095933,0.0,175884800.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-16,0.088485,0.0,0.088923,0.088485,0.088923,0.0,105728000.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-17,0.090676,0.0,0.091114,0.090676,0.090676,0.0,86441600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-18,0.093304,0.0,0.093742,0.093304,0.093304,0.0,73449600.0,Technology,NaN,AAPL,0.093742,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,130.149994,0.0,130.889999,128.460007,130.369995,0.0,91815000.0,Technology,NaN,AAPL,128.745998,126.600665,0.983337,126.600665,2.102349,130.805364,122.395967
2021-06-17,131.789993,0.0,132.550003,129.649994,129.800003,0.0,96721700.0,Technology,NaN,AAPL,129.881996,126.929998,0.977272,126.929998,2.494541,131.919080,121.940917
2021-06-18,130.460007,0.0,131.509995,130.240005,130.710007,0.0,108787300.0,Technology,NaN,AAPL,130.503998,127.275332,0.975260,127.275332,2.605871,132.487074,122.063590


In [8]:
stock_df.ticker.unique()

array(['DVN', 'TER', 'FLT', 'SWKS', 'MSCI', 'WBA', 'DTE', 'DISH', 'NOV',
       'PEG', 'REG', 'LOW', 'CERN', 'NOC', 'BXP', 'ALLE', 'PENN', 'DPZ',
       'CMA', 'PWR', 'HOLX', 'WDC', 'LEG', 'EMR', 'WEC', 'SHW', 'AMAT',
       'NDAQ', 'HES', 'XOM', 'BEN', 'DISCK', 'PTC', 'RF', 'MAR', 'PEAK',
       'EL', 'ALL', 'PAYX', 'AAL', 'MS', 'APA', 'CSX', 'ZION', 'SLB',
       'DLTR', 'WYNN', 'PKG', 'BRK-B', 'PGR', 'BLK'], dtype=object)

In [102]:
fig = go.Figure()

df = stock_df[stock_df['ticker']=='ALL']
df = df.set_index('Date')
    
data = [go.Scatter(x=df.index,
                         y=df['Close'],
                        line={"color": "#228B22"},
                        mode="lines")]

layout = dict(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date"
    )
)

fig = go.FigureWidget(data=data, layout=layout)

fig.update_layout(layout)
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - 1, in_view.High.max() + 1]

fig.layout.on_change(zoom, 'xaxis.range')


fig


FigureWidget({
    'data': [{'line': {'color': '#228B22'},
              'mode': 'lines',
              'type'…

In [104]:
fig2 = go.Figure()

data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])]

layout = dict(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])            
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date",
    ),
    showlegend=False,
    yaxis=dict(autorange = True,fixedrange= False)
)

fig2 = go.FigureWidget(data=data, layout=layout)

fig.update_layout(clickmode='select')

@app.callback(
    Output('relayout-data', 'children'),
    Input('basic-interactions', 'relayoutData'))
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)


def zoom(layout, xrange):
    in_view = df.loc[fig2.layout.xaxis.range[0]:fig2.layout.xaxis.range[1]]
    fig2.layout.yaxis.range = [in_view.Low.min() - 1, in_view.High.max() + 1]

fig2.layout.on_change(zoom, 'xaxis.range')


fig2

NameError: name 'Output' is not defined

In [36]:
fig.layout

Layout({
    'showlegend': False,
    'template': '...',
    'xaxis': {'anchor': 'y',
              'domain': [0.0, 1.0],
              'matches': 'x2',
              'rangeselector': {'buttons': [{'count': 1, 'label': '1m', 'step': 'month', 'stepmode': 'backward'},
                                            {'count': 6, 'label': '6m', 'step': 'month', 'stepmode': 'backward'},
                                            {'count': 1, 'label': 'YTD', 'step': 'year', 'stepmode': 'todate'},
                                            {'count': 1, 'label': '1y', 'step': 'year', 'stepmode': 'backward'},
                                            {'step': 'all'}]},
              'rangeslider': {'visible': False},
              'showticklabels': False,
              'type': 'date'},
    'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0]},
    'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0]},
    'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.425]}
})

In [114]:
df.loc['2020-12-11'].High

122.3689794627517

In [78]:
df

,Close,Date,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
3559733,0.100751,1980-12-12,0.0,0.101189,0.100751,0.100751,0.0,469033600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559734,0.095495,1980-12-15,0.0,0.095933,0.095495,0.095933,0.0,175884800.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559735,0.088485,1980-12-16,0.0,0.088923,0.088485,0.088923,0.0,105728000.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559736,0.090676,1980-12-17,0.0,0.091114,0.090676,0.090676,0.0,86441600.0,Technology,NaN,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3559737,0.093304,1980-12-18,0.0,0.093742,0.093304,0.093304,0.0,73449600.0,Technology,NaN,AAPL,0.093742,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569946,130.149994,2021-06-16,0.0,130.889999,128.460007,130.369995,0.0,91815000.0,Technology,NaN,AAPL,128.745998,126.600665,0.983337,126.600665,2.102349,130.805364,122.395967
3569947,131.789993,2021-06-17,0.0,132.550003,129.649994,129.800003,0.0,96721700.0,Technology,NaN,AAPL,129.881996,126.929998,0.977272,126.929998,2.494541,131.919080,121.940917
3569948,130.460007,2021-06-18,0.0,131.509995,130.240005,130.710007,0.0,108787300.0,Technology,NaN,AAPL,130.503998,127.275332,0.975260,127.275332,2.605871,132.487074,122.063590
3569949,132.300003,2021-06-21,0.0,132.410004,129.210007,130.300003,0.0,79576500.0,Technology,NaN,AAPL,130.867999,127.787999,0.976465,127.787999,2.793728,133.375455,122.200543


In [58]:
in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
in_view

,Close,Date,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband


In [37]:
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]

fig.layout.on_change(zoom, 'xaxis.range')

In [38]:
fig

FigureWidget({
    'data': [{'line': {'color': '#228B22'},
              'mode': 'lines',
              'type'…

In [101]:
from plotly.subplots import make_subplots
fig = make_subplots(shared_xaxes=True ,rows=2, cols=1)

fig.add_trace(
    go.Scatter(x=df.index,
        y=df['Close'],
        line={"color": "#228B22"},
        mode="lines"),
    row=1, col=1
)

fig.add_trace(
    go.Candlestick(x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close']),
    row=2, col=1
)

fig.update_layout(layout)
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - 1, in_view.High.max() + 1]

fig.layout.on_change(zoom, 'xaxis.range')

fig.layout.on_change(lambda obj, xrange, yrange: print("%s-%s" % (xrange, yrange)),\
                     ('xaxis', 'range'), ('yaxis', 'range'))

# fig.update_yaxes(range=[100,130])

fig

In [60]:
import json

In [106]:
app = JupyterDash(__name__,suppress_callback_exceptions=True,external_stylesheets=[dbc.themes.SUPERHERO])

server = app.server

app.layout = html.Div([
    
    dcc.Dropdown(id='data_filter',
                            options=[{'label': '', 'value': ''}],
                            value='CSCO'),
    
    dcc.Graph(id='chart-1',style=chart_style),
    dcc.Graph(id='chart-2',style=chart_style),
    
    html.Div([
            dcc.Markdown("""
                **Zoom and Relayout Data**

                Click and drag on the graph to zoom or click on the zoom
                buttons in the graph's menu bar.
                Clicking on legend items will also fire
                this event.
            """),
            html.Pre(id='relayout-data'),
        ], className='three columns')
    
    ])


@app.callback(dash.dependencies.Output('chart-1','figure'),
             [dash.dependencies.Input('data_filter','value'),])
#               dash.dependencies.Input('chart-1', 'relayoutData')])

# Step 3: Define the graph with plotly express
def update_ticker(value):
    
    fig = go.Figure()
    
    layout2 = dict(showlegend=False,yaxis=dict(
                       autorange = True,
                       fixedrange= False)
                  )
    
#     try:
#         mask = (df.index > new_layout["xaxis.range[0]"]) & (df.index <= new_layout["xaxis.range[1]"])
#         tick_df = df[mask]
#     except:
#         tick_df = df

    tick_df = df

    fig.add_trace(
        go.Scatter(x=tick_df.index,
            y=tick_df['Close'],
            line={"color": "#228B22"},
            mode="lines")
    )

    fig.update_layout(layout)
    fig.update_layout(clickmode='select')
    fig.update_layout(transition_duration=400)
    
    return fig

@app.callback(dash.dependencies.Output('chart-2','figure'),
             [dash.dependencies.Input('data_filter','value'),
              dash.dependencies.Input('chart-1', 'relayoutData')])
# Step 3: Define the graph with plotly express
def update_candlestick(value,new_layout):
    
    fig = go.Figure()
    
    layout2 = dict(showlegend=False)
    
    try:
        mask = (df.index > new_layout["xaxis.range[0]"]) & (df.index <= new_layout["xaxis.range[1]"])
        can_df = df[mask]
    except:
        can_df = df

    fig.add_trace(
        go.Candlestick(x=can_df.index,
        open=can_df['Open'],
        high=can_df['High'],
        low=can_df['Low'],
        close=can_df['Close'])
    )

    fig.update_layout(layout2)
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(clickmode='select')
#     fig.update_layout(transition_duration=400) 

    
    return fig

@app.callback(
    dash.dependencies.Output('relayout-data', 'children'),
    [dash.dependencies.Input('chart-1', 'relayoutData')])
def display_relayout_data(relayoutData):
    
    return json.dumps(relayoutData, indent=2)

if __name__ == '__main__':
    app.run_server(port=8057)

Dash app running on http://127.0.0.1:8057/


In [158]:
df.Close.rolling(window=60).mean()

Date
1993-06-03           NaN
1993-06-04           NaN
1993-06-07           NaN
1993-06-08           NaN
1993-06-09           NaN
                 ...    
2021-06-16    127.022995
2021-06-17    127.278761
2021-06-18    127.476289
2021-06-21    127.711666
2021-06-22    127.930839
Name: Close, Length: 7065, dtype: float64

In [159]:
df['60 Day MA'] = df.Close.rolling(window=60).mean()

In [160]:
df

,Close,Dividends,High,Low,Open,Stock Splits,Volume,sector,test,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband,60 Day MA
Date,,,,,,,,,,,,,,,,,,
1993-06-03,6.973323,0.0,7.121692,6.884302,7.062344,0.0,56603000.0,Financial Services,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-06-04,6.795280,0.0,6.973322,6.795280,6.913975,0.0,12644400.0,Financial Services,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-06-07,6.676585,0.0,6.765606,6.646911,6.706259,0.0,10043800.0,Financial Services,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-06-08,6.587565,0.0,6.706260,6.557891,6.676586,0.0,6974600.0,Financial Services,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-06-09,6.706260,0.0,6.765608,6.617239,6.617239,0.0,4043200.0,Financial Services,NaN,ALL,6.747803,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,130.440002,0.0,131.630005,129.660004,130.500000,0.0,1880900.0,Financial Services,NaN,ALL,131.334000,134.125704,1.021257,134.125704,2.331631,138.788966,129.462442,127.022995
2021-06-17,126.550003,0.0,131.000000,126.169998,131.000000,0.0,1953500.0,Financial Services,NaN,ALL,130.362000,133.577365,1.024665,133.577365,3.030446,139.638257,127.516473,127.278761
2021-06-18,124.040001,0.0,125.750000,123.820000,125.220001,0.0,3910600.0,Financial Services,NaN,ALL,128.824001,132.813312,1.030967,132.813312,3.845921,140.505155,125.121470,127.476289


In [70]:
help(dcc.Graph)

Help on class Graph in module dash_core_components.Graph:

class Graph(dash.development.base_component.Component)
 |  Graph(id=undefined, responsive=undefined, clickData=undefined, clickAnnotationData=undefined, hoverData=undefined, clear_on_unhover=undefined, selectedData=undefined, relayoutData=undefined, extendData=undefined, restyleData=undefined, figure=undefined, style=undefined, className=undefined, animate=undefined, animation_options=undefined, config=undefined, loading_state=undefined, **kwargs)
 |  
 |  A Graph component.
 |  Graph can be used to render any plotly.js-powered data visualization.
 |  
 |  You can define callbacks based on user interaction with Graphs such as
 |  hovering, clicking or selecting
 |  
 |  Keyword arguments:
 |  - id (string; optional): The ID of this component, used to identify dash components
 |  in callbacks. The ID needs to be unique across all of the
 |  components in an app.
 |  - responsive (a value equal to: true, false, 'auto'; default 'a

In [125]:
df.index.max()

Timestamp('2021-06-22 00:00:00')

In [129]:
from datetime import datetime, timedelta
datetime.today().timestamp()

1625326458.268834

In [148]:
df.index.max()

Timestamp('2021-06-22 00:00:00')

In [153]:
app = JupyterDash(__name__,suppress_callback_exceptions=True,external_stylesheets=[dbc.themes.SUPERHERO])

server = app.server

app.layout = html.Div([
    
    html.Button('7 Days', id='btn-nclicks-1',n_clicks=0),
    html.Button('30 Days', id='btn-nclicks-2',n_clicks=0),
    html.Div(id='container-button-timestamp'),
    dcc.Graph(id='chart-1',style=chart_style),
    dcc.Graph(id='chart-2',style=chart_style)
])


@app.callback(
    dash.dependencies.Output('container-button-timestamp', 'children'),
    [dash.dependencies.Input('btn-nclicks-1', 'n_clicks'),
     dash.dependencies.Input('btn-nclicks-2', 'n_clicks')])

def update_output(btn1,btn2):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    msg = ''
    
    if 'btn-nclicks-1' in changed_id:
        msg = 'Button 1 was most recently clicked'
    elif 'btn-nclicks-2' in changed_id:
        msg = 'Button 2 was most recently clicked'

    return html.Div(msg)


@app.callback(dash.dependencies.Output('chart-1','figure'),
             [dash.dependencies.Input('btn-nclicks-1', 'n_clicks'),
             dash.dependencies.Input('btn-nclicks-2', 'n_clicks')])
#               dash.dependencies.Input('chart-1', 'relayoutData')])

# Step 3: Define the graph with plotly express
def update_ticker(btn1,btn2):
    
    fig = go.Figure()
    
    layout2 = dict(showlegend=False,yaxis=dict(
                       autorange = True,
                       fixedrange= False)
                  )
    
    
#     try:
#         mask = (df.index > new_layout["xaxis.range[0]"]) & (df.index <= new_layout["xaxis.range[1]"])
#         tick_df = df[mask]
#     except:
#         tick_df = df

    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'btn-nclicks-1' in changed_id:
        tick_df = df[df.index >= df.index.max()-timedelta(days=7)]
    elif 'btn-nclicks-2' in changed_id:
        tick_df = df[df.index >= df.index.max()-timedelta(days=30)]
    else:
        tick_df = df

    fig.add_trace(
        go.Scatter(x=tick_df.index,
            y=tick_df['Close'],
            line={"color": "#228B22"},
            mode="lines")
    )

    fig.update_layout(layout2)
    fig.update_layout(clickmode='select')
    fig.update_layout(transition_duration=400)
    
    return fig

@app.callback(dash.dependencies.Output('chart-2','figure'),
             [dash.dependencies.Input('chart-1', 'relayoutData')])
# Step 3: Define the graph with plotly express
def update_candlestick(new_layout):
    
    fig = go.Figure()
    
    layout2 = dict(showlegend=False)
    
    try:
        mask = (df.index > new_layout["xaxis.range[0]"]) & (df.index <= new_layout["xaxis.range[1]"])
        can_df = df[mask]
    except:
        can_df = df

    fig.add_trace(
        go.Candlestick(x=can_df.index,
        open=can_df['Open'],
        high=can_df['High'],
        low=can_df['Low'],
        close=can_df['Close'])
    )

    fig.update_layout(layout2)
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(clickmode='select')
#     fig.update_layout(transition_duration=400) 

    
    return fig


if __name__ == '__main__':
    app.run_server(debug=True,port=8055)

Dash app running on http://127.0.0.1:8055/
